# Import bokeh

In [6]:
import os,sys,logging, urllib
import urllib.request
from pprint import pprint

# this may be dangerous, only for local testing/debugging
os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*"

# if you want more debug info
os.environ["BOKEH_LOG_LEVEL"]="debug" 

import bokeh
import bokeh.io 
import bokeh.io.notebook 
import bokeh.models.widgets  
import bokeh.core.validation
import bokeh.plotting
import bokeh.core.validation.warnings
import bokeh.layouts

bokeh.core.validation.silence(bokeh.core.validation.warnings.EMPTY_LAYOUT, True)
bokeh.core.validation.silence(bokeh.core.validation.warnings.FIXED_SIZING_MODE,True)
bokeh.io.output_notebook()

# in JypyterLab/JupyterHub we need to tell what is the proxy url
# see https://docs.bokeh.org/en/3.0.3/docs/user_guide/output/jupyter.html
# example: 
USE_PUBLIC_IP=False

# change this if you need ssh-tunneling
# see https://github.com/sci-visus/OpenVisus/blob/master/docs/ssh-tunnels.md
SSH_TUNNELS=None

# //////////////////////////////////////////////////////////////////////////////////////
def ShowApp(app):
    
    if SSH_TUNNELS:
        notebook_port,bokeh_port=SSH_TUNNELS
        print(f"ShowApp, enabling ssh tunnels notebook_port={notebook_port} bokeh_port={bokeh_port}")
        bokeh.io.notebook.show_app(app, bokeh.io.notebook.curstate(), f"http://127.0.0.1:{notebook_port}", port = bokeh_port) 
        
    elif USE_PUBLIC_IP:
        
        # retrieve public IP (this is needed for front-end browser to reach bokeh server)
        public_ip = urllib.request.urlopen('https://ident.me').read().decode('utf8')
        print(f"public_ip={public_ip}")    
        
        if "JUPYTERHUB_SERVICE_PREFIX" in os.environ:

            def GetJupyterHubNotebookUrl(port):
                if port is None:
                    ret=public_ip
                    print(f"GetJupyterHubNotebookUrl port={port} returning {ret}")
                    return ret
                else:
                    ret=f"https://{public_ip}{os.environ['JUPYTERHUB_SERVICE_PREFIX']}proxy/{port}"
                    print(f"GetJupyterHubNotebookUrl port={port} returning {ret}")
                    return ret     

            bokeh.io.show(app, notebook_url=GetJupyterHubNotebookUrl)
            
        else:
            # need the port (TODO: test), I assume I will get a non-ambiguos/unique port
            import notebook.notebookapp
            ports=list(set([it['port'] for it in notebook.notebookapp.list_running_servers()]))
            assert(len(ports)==1)
            port=ports[0]
            notebook_url=f"{public_ip}:{port}" 
            print(f"bokeh.io.show(app, notebook_url='{notebook_url}')")
            bokeh.io.show(app, notebook_url=notebook_url)
    else:
        bokeh.io.show(app) 
      
# //////////////////////////////////////////////////////////////////////////////////////
def TestBokehApp(doc):
    fig = bokeh.plotting.figure(title="Multiple line example", x_axis_label="x", y_axis_label="y",height=200)
    fig.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5])
    grid=bokeh.layouts.grid(children=[fig], nrows=1, ncols=1, sizing_mode='stretch_width') 
    main_layout=bokeh.layouts.column(children=[],sizing_mode='stretch_width')
    main_layout.children.append(grid)  
    button = bokeh.models.widgets.Button(label="Bokeh is working", sizing_mode='stretch_width')
    button.on_click(lambda: print("Bokeh on_click event is working"))     
    main_layout.children.append(button)
    doc.add_root(main_layout)  

ShowApp(TestBokehApp)

Loading BokehJS ...

Bokeh on_click event is working


# Import OpenVisus

In [7]:
import os,sys,logging, urllib,time,types,datetime,threading,multiprocessing,queue,random,copy,math, types,argparse,logging

os.environ["VISUS_NETSERVICE_VERBOSE"]="0"
os.environ["VISUS_CPP_VERBOSE"       ]="0"
os.environ["VISUS_DASHBOARDS_VERBOSE"]="0"

sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")
import OpenVisus as  ov
from OpenVisus.dashboards import Slice,Slices,DiscreteSlider
# ov.SetupLogger(logging.getLogger("OpenVisus"))
print("OpenVisus imported")

OpenVisus imported


# Show bokeh dashboard

In [18]:
# ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
def MyApp(doc):

    # profile=sealstorage
    os.environ["AWS_ACCESS_KEY_ID"]="any"
    os.environ["AWS_SECRET_ACCESS_KEY"]="any"
    os.environ["AWS_ENDPOINT_URL"]="https://maritime.sealstorage.io/api/v0/s3"

    urls=[f"https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/idx_arco/face{zone}/u_face_{zone}_depth_52_time_0_10269.idx?cached=1" for zone in range(6)]
    palette,palette_range="Turbo256",(-30,60)
    field=None
    logic_to_pixel=[(0.0,1.0), (0.0,1.0), (0.0,10.0)]

    # urls=["https://maritime.sealstorage.io/api/v0/s3/utah/nasa/dyamond/mit_output/llc2160_arco/visus.idx?cached=1"]
    # palette,palette_range="Turbo256",(-1.3,1.7)
    # field=None
    # logic_to_pixel=[(0.0,1.0), (0.0,1.0), (0.0,20.0)]

    # urls=["http://atlantis.sci.utah.edu/mod_visus?dataset=cmip6_cm2&cached=idx"]
    # palette,palette_range="Turbo256",(-1.3,1.7)
    # field="ssp585_tasmax"
    # logic_to_pixel=[(0.0,1.0), (0.0,1.0), (0.0,20.0)]

    ov.dashboards.DIRECTIONS=[('0','Long'),('1','Lat'),('2','Depth')]

    slices=Slices(show_options=["num_views","palette","timestep","timestep_delta","field","quality","num_refinements","!direction","!offset","play-button", "play-msec"])
    slices.logic_to_pixel=logic_to_pixel
    slices.slice_show_options=["direction","offset","status_bar"]

    db=ov.LoadDataset(urls[0])

    slices.setDataset(db)
    slices.setNumberOfViews(3)
    slices.setQuality(-3)
    slices.setNumberOfRefinements(3)
    slices.setPalette(palette, palette_range=palette_range) 

    timesteps=db.getTimesteps()
    N=100
    if len(timesteps)>100*N:
        slices.setTimestepDelta(100)
    elif len(timesteps)>50*N:
        slices.setTimestepDelta(50)
    elif len(timesteps)>10*N:
        slices.setTimestepDelta(10)
    elif len(timesteps)>5*N:
        slices.setTimestepDelta(5)
    elif len(timesteps)>2*N:
        slices.setTimestepDelta(2)
    else:
        slices.setTimestepDelta(1)

    if field is not None:
        slices.setField(field)

    # add the zone zone
    if len(urls)>0:
        url = bokeh.models.Select(title="Zone", options=[str(I) for I in range(len(urls))],width=100) 
        def onUrlChanged(attr, old, new):
            nonlocal slices
            url=urls[int(new)]
            print(f"Setting url={url}")
            db=ov.LoadDataset(url)
            slices.setDataset(db,compatible=True)
        url.on_change("value",onUrlChanged)
        
        first_row=slices.layout.children[0]
        first_row.children.insert(0,url)

    main_layout=bokeh.models.Column(
        slices.layout,
        sizing_mode='stretch_width',
        height=800
    )

    doc.add_root(main_layout)
    
     # TODO: without this it does not work
    def LazySetDataset(evt=None):
        slices.setNumberOfViews(3)         
    
    doc.add_timeout_callback(LazySetDataset ,100) 

ShowApp(MyApp)